In [2]:
#requires Biopython library
#!pip install biopython
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pandas as pd
import regex
import statistics
import os
import glob

def linker_counting(data,fasta):
    records = list(SeqIO.parse(fasta, "fasta"))
    fastas={'id':[],'seq':[],'name':[]}
    for i in range(len(records)):
        fastas['id'].append(records[i].id.split(sep='|')[0])
        fastas['seq'].append(str(records[i].seq))
        fastas['name'].append(str(records[i].id.split(sep='|')[0]))
    fasta_df=pd.DataFrame(fastas)

    BTB_seq=[]
    BTB_names=[]
    for index,protein in data.iterrows():
            C2H2=0
            domains = protein['Matches'].split(sep=',')
            sequence=list(fasta_df[fasta_df['id']==protein['Accession']]['seq'])
            name=list(fasta_df[fasta_df['id']==protein['Accession']]['name'])
            btbs={
                 'start': [],
                 'stop': [],
                   }
            for domain in domains:
                btb=domain.split(sep='..')
                btbs['start'].append(btb[0])
                btbs['stop'].append(btb[1])
            btbs=pd.DataFrame(btbs)
            for btb in range(len(btbs)):
                if (int(btbs['stop'][btb]) - int(btbs['start'][btb]))>220:
                    BTB_seq.append(str(sequence)[(int(btbs['start'][btb])):(int(btbs['stop'][btb]))])
                    BTB_names.append(str(name)[2:-2]+"_"+str(btb))
    return BTB_seq,BTB_names
#function for counting conserved linkers in tsv files
names=[]
clust =[]
linker_seqs=[]
ofile = open('1433_all_Interpro_tsv.fasta', "w")
for file in glob.glob("*.tsv"):
        data =pd.read_table(file)
        name=os.path.basename(file)
        base_name=name.split(sep='.')[0]
        fasta=os.path.join(base_name+'.fasta')
        #clust.append(linker_counting(data,fasta))
        linker_seqs.append(linker_counting(data,fasta))
        names.append(base_name)
        linker_sequences,cluster_names = linker_counting(data,fasta)
        ids=[]
        for i in range(len(linker_sequences)):
            ids.append(i)
        #record = SeqRecord(Seq(linker_sequences),id=ids,)
        fasta_name=os.path.join(base_name+'_clusters.fasta')
        for i in range(len(linker_sequences)):
            ofile.write(">" + cluster_names[i] + "\n" +linker_sequences[i] + "\n")
        #do not forget to close it
        #with open(fasta_name, "w") as output_handle:
        #SeqIO.write(linker_sequences, fasta_name, "fasta")
ofile.close()
        #for file in glob.glob("*.tsv"):
#    data =pd.read_table(file)
#    name=os.path.basename(file)
#    base_name=name.split(sep='.')[0]
#    fasta=os.path.join(base_name+'.fasta')
#    clust.append(linker_counting_tsv(data,fasta))
#    names.append(base_name)
pd.DataFrame(index=names,data=linker_seqs)

,0,1
1433_plants_Interpro,['MSTEKERETHVYTAKLSEQAERYDEMVECMKKVAKLDVELSVDE...,"[A0A022Q774_0, A0A022QMM2_0, A0A022QQK6_0, A0A..."
